In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-in-india/StatewiseTestingDetails.csv
/kaggle/input/covid19-in-india/covid_vaccine_statewise.csv
/kaggle/input/covid19-in-india/covid_19_india.csv


# Let's take a look at data.

In [2]:
Covid_Testing = pd.read_csv('/kaggle/input/covid19-in-india/StatewiseTestingDetails.csv')
Covid_Testing.head()

,Date,State,TotalSamples,Negative,Positive
0,2020-04-17,Andaman and Nicobar Islands,1403.0,1210,12.0
1,2020-04-24,Andaman and Nicobar Islands,2679.0,NaN,27.0
2,2020-04-27,Andaman and Nicobar Islands,2848.0,NaN,33.0
3,2020-05-01,Andaman and Nicobar Islands,3754.0,NaN,33.0
4,2020-05-16,Andaman and Nicobar Islands,6677.0,NaN,33.0


In [3]:
Covid_Testing['State'].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli and Daman and Diu',
       'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
       'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh',
       'Lakshadweep', 'Madhya Pradesh', 'Maharashtra', 'Manipur',
       'Meghalaya', 'Mizoram', 'Nagaland', 'Odisha', 'Puducherry',
       'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana',
       'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal'],
      dtype=object)

In [4]:
print(Covid_Testing['Date'].min())
print(Covid_Testing['Date'].max())

2020-04-01
2021-08-10


#  Handle Date Column

In [5]:
def transform_to_datetime(column):
    Covid_Testing[column] = pd.to_datetime(Covid_Testing[column])

In [6]:
for i in ['Date']:
    transform_to_datetime(i)

In [7]:
 Covid_Testing.dtypes

Date            datetime64[ns]
State                   object
TotalSamples           float64
Negative                object
Positive               float64
dtype: object

In [8]:
Covid_Summary = pd.read_csv('/kaggle/input/covid19-in-india/covid_19_india.csv')
Covid_Summary.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3


# Import required libraries

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [10]:
Covid_Summary.isnull().sum()

Sno                         0
Date                        0
Time                        0
State/UnionTerritory        0
ConfirmedIndianNational     0
ConfirmedForeignNational    0
Cured                       0
Deaths                      0
Confirmed                   0
dtype: int64

In [11]:
Covid_Summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18110 entries, 0 to 18109
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sno                       18110 non-null  int64 
 1   Date                      18110 non-null  object
 2   Time                      18110 non-null  object
 3   State/UnionTerritory      18110 non-null  object
 4   ConfirmedIndianNational   18110 non-null  object
 5   ConfirmedForeignNational  18110 non-null  object
 6   Cured                     18110 non-null  int64 
 7   Deaths                    18110 non-null  int64 
 8   Confirmed                 18110 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.2+ MB


In [12]:
Covid_Summary = Covid_Summary.drop(['Sno','ConfirmedIndianNational', 'ConfirmedForeignNational'], axis=1)
Covid_Summary.head()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed
0,2020-01-30,6:00 PM,Kerala,0,0,1
1,2020-01-31,6:00 PM,Kerala,0,0,1
2,2020-02-01,6:00 PM,Kerala,0,0,2
3,2020-02-02,6:00 PM,Kerala,0,0,3
4,2020-02-03,6:00 PM,Kerala,0,0,3


In [13]:
Covid_Summary['Active'] = Covid_Summary['Confirmed'] - Covid_Summary['Cured'] - Covid_Summary['Deaths']
Covid_Summary.tail()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed,Active
18105,2021-08-11,8:00 AM,Telangana,638410,3831,650353,8112
18106,2021-08-11,8:00 AM,Tripura,77811,773,80660,2076
18107,2021-08-11,8:00 AM,Uttarakhand,334650,7368,342462,444
18108,2021-08-11,8:00 AM,Uttar Pradesh,1685492,22775,1708812,545
18109,2021-08-11,8:00 AM,West Bengal,1506532,18252,1534999,10215


In [14]:
Covid_Summary['Date'] = pd.to_datetime(Covid_Summary['Date'])

In [15]:
Covid_Summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18110 entries, 0 to 18109
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  18110 non-null  datetime64[ns]
 1   Time                  18110 non-null  object        
 2   State/UnionTerritory  18110 non-null  object        
 3   Cured                 18110 non-null  int64         
 4   Deaths                18110 non-null  int64         
 5   Confirmed             18110 non-null  int64         
 6   Active                18110 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 990.5+ KB


In [16]:
india_cases = Covid_Summary[Covid_Summary['Date'] == Covid_Summary['Date'].max()].copy().fillna(0)
india_cases.index = india_cases["State/UnionTerritory"]
india_cases = india_cases.drop(['State/UnionTerritory', 'Time','Date'], axis=1)

In [17]:
india_cases.head()

,Cured,Deaths,Confirmed,Active
State/UnionTerritory,,,,
Andaman and Nicobar Islands,7412,129,7548,7
Andhra Pradesh,1952736,13564,1985182,18882
Arunachal Pradesh,47821,248,50605,2536
Assam,559684,5420,576149,11045
Bihar,715352,9646,725279,281


In [18]:
df = pd.DataFrame(pd.to_numeric(india_cases.sum())).transpose()
df.style.background_gradient(cmap='BuGn',axis=1)

,Cured,Deaths,Confirmed,Active
0,31220981,429179,32036511,386351


In [19]:
Trend = Covid_Summary.groupby(['Date'])['Confirmed', 'Deaths','Cured',].sum().reset_index()
Trend.head()

,Date,Confirmed,Deaths,Cured
0,2020-01-30,1,0,0
1,2020-01-31,1,0,0
2,2020-02-01,2,0,0
3,2020-02-02,3,0,0
4,2020-02-03,3,0,0


In [20]:
fig = go.Figure(go.Bar(x= Trend.Date, y= Trend.Cured, name='Recovered'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Deaths, name='Deaths'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Confirmed, name='Confirmed'))

fig.update_layout(barmode='stack',legend_orientation="h",legend=dict(x= 0.3, y=1.1),
                 paper_bgcolor='white',
                 plot_bgcolor = "white",)
fig.show()

In [21]:
import plotly.express as px

In [22]:
def horizontal_bar_chart(df, x, y, title, x_label, y_label, color):
    fig = px.bar(df, x=x, y=y, orientation='h', title=title, 
                 labels={x.name: x_label,
                         y.name: y_label}, color_discrete_sequence=[color])
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [23]:
top_10_death_states = india_cases.sort_values('Deaths',ascending = False)[:10]
horizontal_bar_chart(top_10_death_states,top_10_death_states.Deaths, top_10_death_states.index,
                     'Top 10 States with most deaths','Number of deaths(In Thousands)','State Name','Orange')

In [24]:
top_10_confirmed_states = india_cases.sort_values('Confirmed', ascending=False)[:10]
horizontal_bar_chart(top_10_confirmed_states,top_10_confirmed_states.Confirmed, top_10_confirmed_states.index,
            'Top 10 Indian States (Confirmed Cases)', 'Number of Confirmed cases (in Thousands)','States Name','blue')


In [25]:
top_10_recovered_states = india_cases.sort_values('Cured', ascending=False)[:10]
horizontal_bar_chart(top_10_recovered_states, top_10_recovered_states.Cured, top_10_recovered_states.index,
                    'Top 10 States (Cured Cases)', 'Number of Cured cases (in Thousands)', 'States Name', 'Purple')

In [26]:
vaccination = pd.read_csv(r"../input/covid19-in-india/covid_vaccine_statewise.csv")
vaccination.tail()

,Updated On,State,Total Doses Administered,Sessions,Sites,First Dose Administered,Second Dose Administered,Male (Doses Administered),Female (Doses Administered),Transgender (Doses Administered),...,18-44 Years (Doses Administered),45-60 Years (Doses Administered),60+ Years (Doses Administered),18-44 Years(Individuals Vaccinated),45-60 Years(Individuals Vaccinated),60+ Years(Individuals Vaccinated),Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Individuals Vaccinated
7840,11/08/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7841,12/08/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7842,13/08/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7843,14/08/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7844,15/08/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
vaccination['Total Vaccinatons'] = vaccination['First Dose Administered']+vaccination['Second Dose Administered']

#Renaming columns
vaccination.rename(columns = {'Updated On':'Date'}, inplace = True)

In [28]:
Maharashtra = vaccination[vaccination["State"]=="Maharashtra"]
fig = px.line(Maharashtra,x="Date",y="Total Vaccinatons",title="Vaccination till date in Maharashtra")  
fig.update_xaxes(rangeslider_visible=True) 

In [29]:
fig = px.line(Maharashtra,x="Date",y="CoviShield (Doses Administered)",title="CoviShield Administered in Mahrashtra")  
fig.update_xaxes(rangeslider_visible=True) 

In [30]:
fig = px.line(Maharashtra,x="Date",y=' Covaxin (Doses Administered)',title="Covaxin Administered in Maharashtra")  
fig.update_xaxes(rangeslider_visible=True) 

For the prediction purpose we have use Prophet library produced by Facebook which is used for Time series Forecasting. Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data

In [31]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
from plotly.offline import iplot, init_notebook_mode
model = Prophet()

In [32]:
Confirmed = Trend.loc[:, ['Date', 'Confirmed']] 
Confirmed.tail()

,Date,Confirmed
555,2021-08-07,31895385
556,2021-08-08,31934455
557,2021-08-09,31969954
558,2021-08-10,31998158
559,2021-08-11,32036511


In [33]:
Cured = Trend.loc[:, ['Date', 'Cured']] 
Cured.tail()

,Date,Cured
555,2021-08-07,31055861
556,2021-08-08,31099771
557,2021-08-09,31139457
558,2021-08-10,31180968
559,2021-08-11,31220981


In [34]:
# Dataframe must have columns "Date" and "Values" with the dates and values  for prophet prediction
Confirmed.columns = ['ds', 'y']
model.fit(Confirmed)

In [35]:
future = model.make_future_dataframe(periods=60) # helper function to extend the dataframe for specified days
future.tail()

,ds
615,2021-10-06
616,2021-10-07
617,2021-10-08
618,2021-10-09
619,2021-10-10


In [36]:
# yhat represents the prediction, while yhat_lower and yhat_upper represent the lower and upper bound
forecast_india_conf = model.predict(future)
forecast_india_conf

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-30,-2.203121e+05,-1.809567e+06,1.469490e+06,-2.203121e+05,-2.203121e+05,21124.184156,21124.184156,21124.184156,21124.184156,21124.184156,21124.184156,0.0,0.0,0.0,-1.991879e+05
1,2020-01-31,-2.167013e+05,-1.962745e+06,1.482158e+06,-2.167013e+05,-2.167013e+05,16030.100764,16030.100764,16030.100764,16030.100764,16030.100764,16030.100764,0.0,0.0,0.0,-2.006712e+05
2,2020-02-01,-2.130905e+05,-1.802352e+06,1.491617e+06,-2.130905e+05,-2.130905e+05,10700.222622,10700.222622,10700.222622,10700.222622,10700.222622,10700.222622,0.0,0.0,0.0,-2.023903e+05
3,2020-02-02,-2.094797e+05,-1.744407e+06,1.489520e+06,-2.094797e+05,-2.094797e+05,4562.834651,4562.834651,4562.834651,4562.834651,4562.834651,4562.834651,0.0,0.0,0.0,-2.049168e+05
4,2020-02-03,-2.058689e+05,-1.888785e+06,1.311346e+06,-2.058689e+05,-2.058689e+05,-4188.545142,-4188.545142,-4188.545142,-4188.545142,-4188.545142,-4188.545142,0.0,0.0,0.0,-2.100574e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,2021-10-06,4.547898e+07,4.359943e+07,4.720068e+07,4.476747e+07,4.623956e+07,-28914.547406,-28914.547406,-28914.547406,-28914.547406,-28914.547406,-28914.547406,0.0,0.0,0.0,4.545007e+07
616,2021-10-07,4.564438e+07,4.386942e+07,4.742452e+07,4.490383e+07,4.643682e+07,21124.184156,21124.184156,21124.184156,21124.184156,21124.184156,21124.184156,0.0,0.0,0.0,4.566550e+07
617,2021-10-08,4.580977e+07,4.407051e+07,4.760691e+07,4.504543e+07,4.663457e+07,16030.100764,16030.100764,16030.100764,16030.100764,16030.100764,16030.100764,0.0,0.0,0.0,4.582580e+07
618,2021-10-09,4.597517e+07,4.413830e+07,4.791978e+07,4.518137e+07,4.682634e+07,10700.222622,10700.222622,10700.222622,10700.222622,10700.222622,10700.222622,0.0,0.0,0.0,4.598587e+07


In [37]:
fig = plot_plotly(model, forecast_india_conf) 

fig.update_layout(template='plotly_white')

iplot(fig) 

In [38]:
model1 = Prophet()
# Dataframe must have columns "Date" and "Values" with the dates and values  for prophet prediction
Cured.columns = ['ds', 'y']
model1.fit(Cured)

future_1 = model1.make_future_dataframe(periods=60) # helper function to extend the dataframe for specified days
future_1.tail()


,ds
615,2021-10-06
616,2021-10-07
617,2021-10-08
618,2021-10-09
619,2021-10-10


In [39]:
# yhat represents the prediction, while yhat_lower and yhat_upper represent the lower and upper bound
forecast_india_cured = model1.predict(future_1)
forecast_india_cured

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-30,-1.437263e+05,-1.500333e+06,1.338121e+06,-1.437263e+05,-1.437263e+05,20692.859570,20692.859570,20692.859570,20692.859570,20692.859570,20692.859570,0.0,0.0,0.0,-1.230335e+05
1,2020-01-31,-1.413917e+05,-1.369133e+06,1.291065e+06,-1.413917e+05,-1.413917e+05,13689.517977,13689.517977,13689.517977,13689.517977,13689.517977,13689.517977,0.0,0.0,0.0,-1.277022e+05
2,2020-02-01,-1.390571e+05,-1.523825e+06,1.125521e+06,-1.390571e+05,-1.390571e+05,7777.857308,7777.857308,7777.857308,7777.857308,7777.857308,7777.857308,0.0,0.0,0.0,-1.312793e+05
3,2020-02-02,-1.367226e+05,-1.378607e+06,1.337868e+06,-1.367226e+05,-1.367226e+05,2123.319301,2123.319301,2123.319301,2123.319301,2123.319301,2123.319301,0.0,0.0,0.0,-1.345992e+05
4,2020-02-03,-1.343880e+05,-1.502850e+06,1.219514e+06,-1.343880e+05,-1.343880e+05,-8339.938716,-8339.938716,-8339.938716,-8339.938716,-8339.938716,-8339.938716,0.0,0.0,0.0,-1.427279e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,2021-10-06,4.473314e+07,4.304842e+07,4.628484e+07,4.392054e+07,4.550097e+07,-22141.705314,-22141.705314,-22141.705314,-22141.705314,-22141.705314,-22141.705314,0.0,0.0,0.0,4.471100e+07
616,2021-10-07,4.490559e+07,4.316965e+07,4.646913e+07,4.406331e+07,4.569491e+07,20692.859570,20692.859570,20692.859570,20692.859570,20692.859570,20692.859570,0.0,0.0,0.0,4.492628e+07
617,2021-10-08,4.507804e+07,4.351427e+07,4.663129e+07,4.420729e+07,4.588947e+07,13689.517977,13689.517977,13689.517977,13689.517977,13689.517977,13689.517977,0.0,0.0,0.0,4.509173e+07
618,2021-10-09,4.525049e+07,4.361561e+07,4.692607e+07,4.436470e+07,4.608144e+07,7777.857308,7777.857308,7777.857308,7777.857308,7777.857308,7777.857308,0.0,0.0,0.0,4.525827e+07


In [40]:
fig = plot_plotly(model1, forecast_india_cured) 

fig.update_layout(template='plotly_white')

iplot(fig) 